In [ ]:
 !pip install -q kaggle
 !pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp37-none-any.whl size=73269 sha256=d482ebf7ed40d494d2b1ec65c729c56d2edce163a510e202209f30bdbef823b3
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ranzcr-clip-catheter-line-classification

100% 11.7G/11.7G [02:19<00:00, 59.2MB/s]
100% 11.7G/11.7G [02:19<00:00, 89.9MB/s]


In [ ]:
!mkdir ranzcr-clip-catheter-line-classification
!unzip ranzcr-clip-catheter-line-classification.zip -d ranzcr-clip-catheter-line-classification

Streaming output truncated to the last 5000 lines.
  inflating: ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.80037468194220420320170944397575295227.jpg  
  inflating: ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.80048407175337306557451404132392019931.jpg  
  inflating: ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.80058141917130703975834534533058558900.jpg  
  inflating: ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.80062845141741671052045396402379570613.jpg  
  inflating: ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.80065354188180228774750311922384866730.jpg  
  inflating: ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.80065596468977553272320950387981297089.jpg  
  inflating: ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.80068774453246950728257986408626689267.jpg  
  inflating: ranzcr-clip-cathete

In [ ]:
!rm -rf ranzcr-clip-catheter-line-classification/*_tfrecords

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
#!pip install wandb
#import wandb
#wandb.init()

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU: {tpu.master()}')
except ValueError:
    tpu = None
    print('Running on GPU')

Running on GPU


In [ ]:
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Finished configuring TPU')
else:
    strategy = tf.distribute.get_strategy()

In [ ]:
REPLICAS = strategy.num_replicas_in_sync
#print(f'Number of replicas: {REPLICAS}')

In [ ]:
WORK_DIR = os.getcwd() + '/ranzcr-clip-catheter-line-classification'
#os.listdir(WORK_DIR)

In [ ]:
train = pd.read_csv(os.path.join(WORK_DIR, 'train.csv'))
#train.head(5)

In [ ]:
train_image_names = WORK_DIR + '/train/' + train['StudyInstanceUID'] + '.jpg'
#train_image_names[0]

In [ ]:
sample_submission = pd.read_csv(os.path.join(WORK_DIR, 'sample_submission.csv'))
#sample_submission.head(5)

In [ ]:
label_columns = sample_submission.columns[1:]
#print(label_columns)

In [ ]:
labels = train[label_columns].values
#labels[0]

In [ ]:
train_annot = pd.read_csv(os.path.join(WORK_DIR, "train_annotations.csv"))
#train_annot.head(5)

In [ ]:
# sample = train.sample(9)
# plt.figure(figsize=(10, 7))
# for ind, image_id in enumerate(sample.StudyInstanceUID):
#     plt.subplot(3, 3, ind + 1)
#     image = image_id + '.jpg'
#     img = cv2.imread(os.path.join(WORK_DIR, "train", image))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     plt.imshow(img)
#     plt.title('Shape: {}'.format(img.shape[:2]))
#     plt.axis("off")
# plt.show()

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * REPLICAS
STEPS_PER_EPOCH = len(train) * 0.8 // BATCH_SIZE
VALIDATION_STEPS = len(train) * 0.2 // BATCH_SIZE
EPOCHS = 30
TARGET_SIZE = 260 #EfficientNet B2
#TARGET_SIZE = 300 #EfficientNet B3
#TARGET_SIZE = 380 #EfficientNet B4

In [ ]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE, TARGET_SIZE), 
                  ext = 'jpg'):
    def decode(path):
        #print(path)
        #with open(path, "rb") as local_file: # <= change here
        #  img = local_file.read()
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

# in this part you can choose any type of augmentation from the ones suggested above
def build_augmenter(with_labels = True):
    def augment(img):
        #img = NeedleAugmentation(img, n_needles=2, dark_needles=False, p=0.5)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 0.9, 1)
        img = tf.image.random_contrast(img, 0.9, 1)
        img = tf.image.random_hue(img, 0.5)
        #img = tf.image.random_saturation(img, 5, 10)
        #img = tf.image.random_saturation(img, 0.9, 1) 
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
(train_img, valid_img, 
 train_labels, valid_labels) = train_test_split(train_image_names, labels, 
                                                train_size = 0.8, 
                                                random_state = 42)

In [ ]:
train_ds = build_dataset(train_img, train_labels, bsize=BATCH_SIZE, cache=False)
valid_ds = build_dataset(valid_img, valid_labels, bsize=BATCH_SIZE, cache=False, shuffle=False, augment=False, repeat=False)

In [ ]:
model_save_EfficientNetB3 = ModelCheckpoint('./EfficientNetB4_best_weights.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_auc', 
                             mode = 'max', verbose = 1)
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", patience=3, min_lr=1e-6, mode='max')

In [ ]:
 with strategy.scope():
    base_model = tf.keras.applications.EfficientNetB2(include_top=False, weights='imagenet', input_shape=(TARGET_SIZE, TARGET_SIZE, 3))
    #base_model = tf.keras.applications.EfficientNetB3(include_top=False, weights='imagenet', input_shape=(TARGET_SIZE, TARGET_SIZE, 3))
    #base_model = tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet', input_shape=(TARGET_SIZE, TARGET_SIZE, 3))
    model = base_model.output
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    #model = tf.keras.layers.Dense(64, activation='relu')(model)
    #model = tf.keras.layers.Dropout(0.5)(model)
    model = tf.keras.layers.Dense(11, activation = "sigmoid")(model)

    model = tf.keras.models.Model(base_model.input, model)
 
    model.compile(optimizer = 'adam',
                  loss = "binary_crossentropy",
                  metrics = [tf.keras.metrics.AUC(multi_label = True)])


71688192/71686520 [==============================] - 1s 0us/step


In [ ]:
history = model.fit(train_ds, epochs=20, validation_data=valid_ds, steps_per_epoch=STEPS_PER_EPOCH, validation_steps=VALIDATION_STEPS, verbose=1, callbacks=[model_save_EfficientNetB3, lr_reducer])

Epoch 1/20
1504/1504 [==============================] - 1440s 939ms/step - loss: 0.0614 - auc: 0.9890 - val_loss: 1.0731 - val_auc: 0.5172

Epoch 00001: val_auc improved from -inf to 0.51723, saving model to ./EfficientNetB4_best_weights.h5
Epoch 2/20
1504/1504 [==============================] - 1416s 942ms/step - loss: 0.0591 - auc: 0.9907 - val_loss: 0.7880 - val_auc: 0.5005

Epoch 00002: val_auc did not improve from 0.51723
Epoch 3/20
1360/1504 [==========================>...] - ETA: 1:59 - loss: 0.0553 - auc: 0.9909